In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-22 01:59:46.309024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 01:59:46.412859: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-22 01:59:46.438458: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [3]:
# SERLU 함수 정의
def serlu(x, lamda = 1.0786, alpha= 2.90427):
    return tf.where(x>=0, x, alpha * x * tf.exp(x))

In [9]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(100, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 14, 14, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 64)        18496     
                                                           

In [10]:
# 옵티마이저 및 학습률 스케줄러 정의
# 옵티마이저 정의
learning_rate = 1e-3

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate


opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate )


model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [11]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [12]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR100", entity="hcim", name='LeNet-serlu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇██████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▃▄▅▆▆▆▆▇▇▇▇█▇█████████
val_loss,█▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.48304
best_epoch,22
best_val_accuracy,0.4137
epoch,22
loss,1.92569
val_accuracy,0.4137


In [ ]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Learning rate: 0.001
Epoch 1/200
388/391 [============================>.] - ETA: 0s - loss: 3.8893 - accuracy: 0.1124

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best)... Done. 0.0s


391/391 [==============================] - 24s 59ms/step - loss: 3.8869 - accuracy: 0.1127 - val_loss: 3.4425 - val_accuracy: 0.1790 - lr: 0.0010
Learning rate: 0.001
Epoch 2/200
389/391 [============================>.] - ETA: 0s - loss: 3.3009 - accuracy: 0.2048

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 56ms/step - loss: 1.0845 - accuracy: 0.6938 - val_loss: 2.1171 - val_accuracy: 0.4850 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 153/200
391/391 [==============================] - 21s 53ms/step - loss: 1.0876 - accuracy: 0.6925 - val_loss: 2.1192 - val_accuracy: 0.4845 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 154/200
389/391 [============================>.] - ETA: 0s - loss: 1.0840 - accuracy: 0.6921

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_020841-fin78d1x/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 57ms/step - loss: 1.0841 - accuracy: 0.6920 - val_loss: 2.1181 - val_accuracy: 0.4851 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 155/200
391/391 [==============================] - 20s 52ms/step - loss: 1.0817 - accuracy: 0.6941 - val_loss: 2.1184 - val_accuracy: 0.4842 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 156/200
391/391 [==============================] - 21s 53ms/step - loss: 1.0822 - accuracy: 0.6941 - val_loss: 2.1186 - val_accuracy: 0.4835 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 157/200
391/391 [==============================] - 21s 54ms/step - loss: 1.0834 - accuracy: 0.6929 - val_loss: 2.1182 - val_accuracy: 0.4846 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 158/200
391/391 [==============================] - 20s 52ms/step - loss: 1.0862 - accuracy: 0.6908 - val_loss: 2.1184 - val_accuracy: 0.4850 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 159/200
391/391 [==============================] - 21s 52ms/step - loss: 1.0835 - a